In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import pandas as pd
from bs4 import BeautifulSoup
import urllib
import re
import requests
import csv
import time
import pyarrow.parquet as pq

In [ ]:

# List to store the data
all_data = []

# Initialize webdriver
driver = webdriver.Firefox()

# Loop through pages
for page in range(0, 1):
    # Navigate to page
    driver.get(f'https://www.wineandco.com/vins#page_type=taxon&category_id=2&pageNumber={page}')
    time.sleep(1)

    # Handle cookie dialog
    try:
        cookie_button = driver.find_element(By.CLASS_NAME, 'm-modal__close')
        cookie_button.click()
    except:
        pass

    # Get product links from the page
    product_links = [x.get_attribute('href') for x in driver.find_elements(By.XPATH,'//div[@class="c-wine"]/a')]

    # Loop through product links
    for link in product_links:
        # Navigate to product page
        driver.get(link)
        time.sleep(1)

        # Extract product information
        product = driver.find_element(By.CLASS_NAME, "l-product")

        # Helper function to extract data
        def extract_data(xpath):
            try:
                data = product.find_element(By.XPATH, xpath).get_attribute('textContent')
            except:
                data = ""
            return data

        # Basic product information
        name = extract_data('//h1[@class="l-product__infos__name"]')
        price = extract_data('//span[@class="price-display"]')
        rating = extract_data('//div[@class="l-product__infos__avist"]/span')

        product_info = driver.find_elements(By.CLASS_NAME,"c-product-characteristics__container")

        # Find the div with class 'c-product-characteristics__container'
        # Product characteristics
        product_div = driver.find_element(By.CLASS_NAME, 'c-product-characteristics__container')
        characteristics = []

            # Extract characteristics information
        for ul in product_div.find_elements(By.TAG_NAME, 'ul'):
            for li in ul.find_elements(By.TAG_NAME, 'li'):
                characteristic = {}

                    # Extract label and text
                label = li.find_element(By.CLASS_NAME, 'c-product-characteristics__label').text
                text = li.find_element(By.CLASS_NAME, 'c-product-characteristics__text').text

                    # Store in dictionary
                characteristic[label] = text
                characteristics.append(characteristic)

            # Store data in dictionary
        data = {'name': name, 'price': price, 'rating': rating, 'characteristics': characteristics}
        all_data.append(data)


# Close the webdriver session
driver.quit()

# Print the list of dictionaries
print(all_data)

IndexError: list index out of range